In [18]:
import sqlite3
import pandas as pd

file_path = 'emissions_medium_granularity.csv'
data = pd.read_csv(file_path)

# Kết nối tới SQLite database (hoặc tạo database nếu chưa tồn tại)
conn = sqlite3.connect('emissions.db')
cursor = conn.cursor()

# Xóa các bảng cũ nếu có (lưu ý: điều này sẽ xóa toàn bộ dữ liệu hiện tại trong các bảng này)
cursor.execute('DROP TABLE IF EXISTS Emissions')
cursor.execute('DROP TABLE IF EXISTS Entities')
cursor.execute('DROP TABLE IF EXISTS Commodities')

# Tạo các bảng mới
cursor.execute('''
    CREATE TABLE Entities (
        EntityID INTEGER PRIMARY KEY AUTOINCREMENT,
        ParentEntity TEXT NOT NULL,
        ParentType TEXT
    )
''')

cursor.execute('''
    CREATE TABLE Commodities (
        CommodityID INTEGER PRIMARY KEY AUTOINCREMENT,
        Commodity TEXT NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE Emissions (
        EmissionID INTEGER PRIMARY KEY AUTOINCREMENT,
        Year INTEGER,
        EntityID INTEGER,
        CommodityID INTEGER,
        ProductionValue REAL,
        ProductionUnit TEXT,
        TotalEmissionsMtCO2e REAL,
        FOREIGN KEY (EntityID) REFERENCES Entities (EntityID),
        FOREIGN KEY (CommodityID) REFERENCES Commodities (CommodityID)
    )
''')

# Chèn dữ liệu vào bảng Entities
entities = data[['parent_entity', 'parent_type']].drop_duplicates()
for index, row in entities.iterrows():
    cursor.execute("INSERT INTO Entities (ParentEntity, ParentType) VALUES (?, ?)", (row['parent_entity'], row['parent_type']))

# Chèn dữ liệu vào bảng Commodities
commodities = data['commodity'].drop_duplicates()
for commodity in commodities:
    cursor.execute("INSERT INTO Commodities (Commodity) VALUES (?)", (commodity,))

# Tạo từ điển để ánh xạ tên thực thể và loại hàng hóa với ID của chúng
entity_map = {row['parent_entity']: cursor.execute("SELECT EntityID FROM Entities WHERE ParentEntity=?", (row['parent_entity'],)).fetchone()[0] for index, row in entities.iterrows()}
commodity_map = {commodity: cursor.execute("SELECT CommodityID FROM Commodities WHERE Commodity=?", (commodity,)).fetchone()[0] for commodity in commodities}

# Chèn dữ liệu vào bảng Emissions
for index, row in data.iterrows():
    cursor.execute('''
        INSERT INTO Emissions (Year, EntityID, CommodityID, ProductionValue, ProductionUnit, TotalEmissionsMtCO2e)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['year'], entity_map[row['parent_entity']], commodity_map[row['commodity']], row['production_value'], row['production_unit'], row['total_emissions_MtCO2e']))

# Commit và đóng kết nối
conn.commit()
conn.close()


In [12]:
import pyodbc
import pandas as pd

# Đọc dữ liệu từ file CSV
file_path = 'emissions_medium_granularity.csv'
data = pd.read_csv(file_path)

# Thay thế bằng thông tin thực tế của bạn trên Azure
server = 'databasefordbm.database.windows.net'
database = 'emissions_db'
username = 'thanhbinh'
password = 'phanbinh@1234'
driver = '{ODBC Driver 18 for SQL Server}'

# Chuỗi kết nối
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

# Kết nối tới Azure SQL Database
conn_azure = pyodbc.connect(conn_str)
cursor_azure = conn_azure.cursor()

# Tạo bảng trong Azure SQL Database
cursor_azure.execute('''
    CREATE TABLE Emissions (
        year INT,
        parent_entity NVARCHAR(100),
        parent_type NVARCHAR(50),
        commodity NVARCHAR(50),
        production_value FLOAT,
        production_unit NVARCHAR(50),
        total_emissions_MtCO2e FLOAT
    )
''')

# Chèn dữ liệu vào bảng trong Azure SQL Database
for index, row in data.iterrows():
    cursor_azure.execute('''
        INSERT INTO Emissions (year, parent_entity, parent_type, commodity, production_value, production_unit, total_emissions_MtCO2e)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))

conn_azure.commit()
cursor_azure.close()
conn_azure.close()


In [16]:
import pyodbc
import pandas as pd

server = 'databasefordbm.database.windows.net'
database = 'emissions_db'
username = 'thanhbinh'
password = 'phanbinh@1234'
driver = '{ODBC Driver 18 for SQL Server}'
# Chuỗi kết nối
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

# Kết nối tới Azure SQL Database
conn_azure = pyodbc.connect(conn_str)

# Đọc dữ liệu từ bảng
query = 'SELECT * FROM Emissions'
data = pd.read_sql(query, conn_azure)

conn_azure.close()

# Hiển thị dữ liệu
print(data.head())

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2488\789381301.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn_azure)


   year                   parent_entity         parent_type    commodity  \
0  1962  Abu Dhabi National Oil Company  State-owned Entity    Oil & NGL   
1  1962  Abu Dhabi National Oil Company  State-owned Entity  Natural Gas   
2  1963  Abu Dhabi National Oil Company  State-owned Entity    Oil & NGL   
3  1963  Abu Dhabi National Oil Company  State-owned Entity  Natural Gas   
4  1964  Abu Dhabi National Oil Company  State-owned Entity    Oil & NGL   

   production_value production_unit  total_emissions_MtCO2e  
0           0.91250  Million bbl/yr                0.363885  
1           1.84325          Bcf/yr                0.134355  
2           1.82500  Million bbl/yr                0.727770  
3           4.42380          Bcf/yr                0.322453  
4           7.30000  Million bbl/yr                2.911079  
